In [1]:
# import packages
import pandas as pd
import os
import glob
import random
import numpy as np
import statistics
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import matplotlib.patches as mpatches

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
from scipy import stats
from scipy.stats import norm

## Data Prep

In [2]:
path = r'/Users/spencersu/Desktop/Efficacy Studies/Archdiocese'
df = pd.read_csv(os.path.join(path, 'Arch 2020-22 growth.csv'))


## clean the columns

df = df.iloc[:,1:]
df = df.rename(columns = {'Student Id [Sensitive]':'student_id', 'Full Name [Sensitive]':'student_name',
                         'School Year':'school_year','School Name':'school_name','Class Grade Level':'grade',
                         'Teacher Name [Sensitive]':'teacher_name','Lesson Type':'lesson_type',
                         'Multiple Choice Score':'reading_score','Word Count In Final Writing':'word_count',
                         'Number of Graded Writing Lessons':'num_graded_writing','Number of Lessons':'num_lesson'})
df['word_count'] = pd.to_numeric(df['word_count'],errors='coerce')
df #6727 rows × 11 columns

,student_id,student_name,school_year,school_name,grade,teacher_name,lesson_type,reading_score,word_count,num_graded_writing,num_lesson
0,1215107,Zoey Todd,2020-2021,Sacred Heart School,3,Alyssa Reyes,Writing Lesson,0.0,10.0,0,1
1,1215107,Zoey Todd,2021-2022,Sacred Heart School,4,Sandra Knight,Benchmark - Baseline Writing,10.0,24.0,0,1
2,1215107,Zoey Todd,2021-2022,Sacred Heart School,4,Sandra Knight,Benchmark - End-of-Year,50.0,28.0,0,1
3,1215107,Zoey Todd,2021-2022,Sacred Heart School,4,Sandra Knight,Benchmark - Leveling,37.5,NaN,0,1
4,1215107,Zoey Todd,2021-2022,Sacred Heart School,4,Sandra Knight,Benchmark - Leveling,NaN,NaN,0,2
...,...,...,...,...,...,...,...,...,...,...,...
6722,1254553,Aaleyah Reyes,2021-2022,St. William School,6,Karen Zaccaria,Benchmark - Leveling,50.0,NaN,0,1
6723,1254553,Aaleyah Reyes,2021-2022,St. William School,6,Karen Zaccaria,Benchmark - Leveling,87.5,NaN,0,1
6724,1254553,Aaleyah Reyes,2021-2022,St. William School,6,Karen Zaccaria,Reading Practice,20.0,NaN,0,1
6725,1254553,Aaleyah Reyes,2021-2022,St. William School,6,Karen Zaccaria,Reading Practice,40.0,NaN,0,3


In [3]:
# df.to_csv(os.path.join(path,'overall graph.csv'))

In [ ]:
## !!!adding all the lessons up might not be right unless we filter out those that reading practice and word count are both nan

---

## Data Overview

### 1. District Level Usage
p.s. Num of students participating (start a lesson), num of lessons, num of graded writing lessons

1. (a) the number of students start a lesson for each lesson type each year (student:benchmark=1:1)

In [4]:
print(df.groupby('school_year')['student_id'].nunique())
print(df.groupby(['school_year','lesson_type'])['student_id'].nunique())


## 1. total number of students starting a lesson increases by a little
## 2. fewer students start boy benchmark lessons, more benchmark leveling and mid-year, 
##    a lot more reading, but almost the same writing

school_year
2020-2021    636
2021-2022    680
Name: student_id, dtype: int64
school_year  lesson_type                 
2020-2021    Benchmark - Baseline Writing    113
             Benchmark - Leveling            148
             Benchmark - Mid-Year             46
             Reading Practice                169
             Writing Lesson                  546
2021-2022    Benchmark - Baseline Writing    334
             Benchmark - End-of-Year          86
             Benchmark - Leveling            196
             Benchmark - Mid-Year            107
             Reading Practice                244
             Writing Lesson                  558
Name: student_id, dtype: int64


1. (b) total number of lessons 

In [5]:
df.groupby('school_year')['num_lesson', 'num_graded_writing'].sum().reset_index()

## total number of lessons and graded writing lessons both increase! 

,school_year,num_lesson,num_graded_writing
0,2020-2021,3342,745
1,2021-2022,4618,1667


2. count the number of lessons and graded writing lessons per student from last to this year 

In [6]:
## only count those who have finished writing (have values in word count)
cntlsn_wc = df[df['word_count'].notna()].groupby(['school_year',
                                                  'student_id',
                                                  'lesson_type'])['num_lesson',
                                                                  'num_graded_writing'].sum()
# cntlsn_wc = cntlsn_wc.to_frame().reset_index()
print(cntlsn_wc.groupby(['school_year','lesson_type'])['num_lesson', 'num_graded_writing'].mean().reset_index())
print(cntlsn_wc.groupby(['school_year'])['num_lesson', 'num_graded_writing'].sum().reset_index())

  school_year                   lesson_type  num_lesson  num_graded_writing
0   2020-2021  Benchmark - Baseline Writing    1.000000            0.000000
1   2020-2021          Benchmark - Mid-Year    1.000000            0.000000
2   2020-2021                Writing Lesson    3.564885            1.872774
3   2021-2022  Benchmark - Baseline Writing    1.003106            0.000000
4   2021-2022       Benchmark - End-of-Year    1.000000            0.000000
5   2021-2022          Benchmark - Mid-Year    1.000000            0.000000
6   2021-2022                Writing Lesson    4.563126            3.234469
  school_year  num_lesson  num_graded_writing
0   2020-2021        1546                 736
1   2021-2022        2781                1614


In [7]:
## check for reading practice and those only start a writing as well
df.groupby(['school_year',
            'student_id',
            'lesson_type'])['num_lesson',
                            'num_graded_writing'].sum().groupby(['school_year',
                                                                 'lesson_type'])['num_lesson',
                                                                                 'num_graded_writing'].mean().reset_index()

,school_year,lesson_type,num_lesson,num_graded_writing
0,2020-2021,Benchmark - Baseline Writing,1.000000,0.000000
1,2020-2021,Benchmark - Leveling,3.182432,0.000000
2,2020-2021,Benchmark - Mid-Year,1.000000,0.000000
3,2020-2021,Reading Practice,4.786982,0.000000
4,2020-2021,Writing Lesson,3.485348,1.364469
5,2021-2022,Benchmark - Baseline Writing,1.005988,0.000000
6,2021-2022,Benchmark - End-of-Year,1.000000,0.000000
7,2021-2022,Benchmark - Leveling,3.280612,0.000000
8,2021-2022,Benchmark - Mid-Year,1.000000,0.000000
9,2021-2022,Reading Practice,2.647541,0.000000


In [8]:
df.groupby(['school_year',
            'student_id'])['num_lesson',
                           'num_graded_writing'].sum().groupby(['school_year'])['num_lesson',
                                                                                'num_graded_writing'].mean().reset_index()

,school_year,num_lesson,num_graded_writing
0,2020-2021,5.254717,1.171384
1,2021-2022,6.791176,2.451471


3. writing lesson usage 

In [9]:
df[df['lesson_type'] == 'Writing Lesson'].groupby('school_year')['num_lesson','num_graded_writing'].sum()

,num_lesson,num_graded_writing
school_year,,
2020-2021,1903,745
2021-2022,2800,1667


### 2. District Level Reading Score & Writing Word Count Growth

1. Avg. Rd Score and WC from 2020-2021 to 2021-2022

In [10]:
print(df.groupby('school_year')['reading_score'].mean())
print(df.groupby('school_year')['word_count'].mean())

## both avg. reading score and word count have improved

school_year
2020-2021    69.235243
2021-2022    74.629646
Name: reading_score, dtype: float64
school_year
2020-2021    170.506903
2021-2022    198.602745
Name: word_count, dtype: float64


2. Avg. Rd Score Across All Lesson Types

In [11]:
df.groupby(['school_year','lesson_type'])['reading_score'].mean()
## Writing lesson multiple choice cores have improved from 72.67 to 81.10, 
## and reading practice does not have a significant change or growth 

school_year  lesson_type                 
2020-2021    Benchmark - Baseline Writing    60.849057
             Benchmark - Leveling            60.405028
             Benchmark - Mid-Year            65.365854
             Reading Practice                67.788462
             Writing Lesson                  72.671429
2021-2022    Benchmark - Baseline Writing    58.181818
             Benchmark - End-of-Year         57.746479
             Benchmark - Leveling            66.763771
             Benchmark - Mid-Year            61.470588
             Reading Practice                66.822222
             Writing Lesson                  81.103911
Name: reading_score, dtype: float64

In [12]:
df.groupby(['school_year','lesson_type'])['word_count'].mean()

school_year  lesson_type                 
2020-2021    Benchmark - Baseline Writing    225.628571
             Benchmark - Leveling                   NaN
             Benchmark - Mid-Year             44.350000
             Reading Practice                       NaN
             Writing Lesson                  169.968023
2021-2022    Benchmark - Baseline Writing    116.241486
             Benchmark - End-of-Year          93.312500
             Benchmark - Leveling                   NaN
             Benchmark - Mid-Year            143.287129
             Reading Practice                       NaN
             Writing Lesson                  216.533333
Name: word_count, dtype: float64

3. word count and reading score improvement from last year to this year for each grade level 

In [13]:
print(df.groupby(['school_year','grade'])['word_count'].mean())
print(df.groupby(['school_year','grade'])['reading_score'].mean())

## for word count, 6th, 7th grade has the biggest improvement, 3rd, 4th, 8th have decline, and 5th is about the same

school_year  grade
2020-2021    3         96.000000
             4         90.133333
             5        110.367089
             6         64.095057
             7        161.731898
             8        231.653968
             13       243.090909
2021-2022    3         78.904255
             4         62.115942
             5        112.205128
             6        188.840278
             7        233.333655
             8        199.042237
Name: word_count, dtype: float64
school_year  grade
2020-2021    3        60.000000
             4        64.285714
             5        61.625000
             6        56.061798
             7        70.257833
             8        76.529642
             13       78.181818
2021-2022    3        71.011236
             4        55.076531
             5        57.484292
             6        69.961302
             7        79.703231
             8        77.577666
Name: reading_score, dtype: float64


### 3. School level usage 

1. the number of students in each school for each lesson type

In [14]:
df.groupby(['school_year','lesson_type', 'school_name'])['student_id'].nunique()

school_year  lesson_type                   school_name                                
2020-2021    Benchmark - Baseline Writing  St. Albert the Great Catholic School            16
                                           St. John Brebeuf School                         73
                                           St. William School                              24
             Benchmark - Leveling          St Gabriel School                               21
                                           St. Albert the Great Catholic School            16
                                           St. John Brebeuf School                         43
                                           St. William School                              68
             Benchmark - Mid-Year          St Gabriel School                               25
                                           St. William School                              21
             Reading Practice              St. Albert the Great Cat

2. total number of lessons and graded writing lessons

In [15]:
df.groupby(['school_year','lesson_type','school_name'])['num_lesson','num_graded_writing'].sum().reset_index()

,school_year,lesson_type,school_name,num_lesson,num_graded_writing
0,2020-2021,Benchmark - Baseline Writing,St. Albert the Great Catholic School,16,0
1,2020-2021,Benchmark - Baseline Writing,St. John Brebeuf School,73,0
2,2020-2021,Benchmark - Baseline Writing,St. William School,24,0
3,2020-2021,Benchmark - Leveling,St Gabriel School,63,0
4,2020-2021,Benchmark - Leveling,St. Albert the Great Catholic School,16,0
5,2020-2021,Benchmark - Leveling,St. John Brebeuf School,189,0
6,2020-2021,Benchmark - Leveling,St. William School,203,0
7,2020-2021,Benchmark - Mid-Year,St Gabriel School,25,0
8,2020-2021,Benchmark - Mid-Year,St. William School,21,0
9,2020-2021,Reading Practice,St. Albert the Great Catholic School,16,0


3. avg number of lessons and graded writing lessons 

In [16]:
sch_cntlsn_left = df.groupby(['school_year','lesson_type', 'school_name'])['student_id'].nunique()
sch_cntlsn_left.to_frame().reset_index()

sch_cntlsn_right = df.groupby(['school_year',
                               'lesson_type',
                               'school_name'])['num_lesson','num_graded_writing'].sum().reset_index()
sch_cntlsn = pd.merge(sch_cntlsn_left, sch_cntlsn_right, on=['school_year','lesson_type', 'school_name'], how='inner')
sch_cntlsn['avg_lsn'] = sch_cntlsn['num_lesson'] / sch_cntlsn['student_id']
sch_cntlsn['avg_graded'] = sch_cntlsn['num_graded_writing'] / sch_cntlsn['student_id']
sch_cntlsn
# sch_cntlsn.to_csv(os.path.join(path, 'school level usage.csv'))

,school_year,lesson_type,school_name,student_id,num_lesson,num_graded_writing,avg_lsn,avg_graded
0,2020-2021,Benchmark - Baseline Writing,St. Albert the Great Catholic School,16,16,0,1.000000,0.000000
1,2020-2021,Benchmark - Baseline Writing,St. John Brebeuf School,73,73,0,1.000000,0.000000
2,2020-2021,Benchmark - Baseline Writing,St. William School,24,24,0,1.000000,0.000000
3,2020-2021,Benchmark - Leveling,St Gabriel School,21,63,0,3.000000,0.000000
4,2020-2021,Benchmark - Leveling,St. Albert the Great Catholic School,16,16,0,1.000000,0.000000
5,2020-2021,Benchmark - Leveling,St. John Brebeuf School,43,189,0,4.395349,0.000000
6,2020-2021,Benchmark - Leveling,St. William School,68,203,0,2.985294,0.000000
7,2020-2021,Benchmark - Mid-Year,St Gabriel School,25,25,0,1.000000,0.000000
8,2020-2021,Benchmark - Mid-Year,St. William School,21,21,0,1.000000,0.000000
9,2020-2021,Reading Practice,St. Albert the Great Catholic School,16,16,0,1.000000,0.000000


In [17]:
sch_cntlsn.pivot(index=['lesson_type','school_name'], 
                 columns='school_year', 
                 values=['num_lesson','num_graded_writing', 'student_id',
                         'avg_lsn', 'avg_graded']).to_csv(os.path.join(path, 'school level usage.csv'))

### Findings:

1. total number of writing lessons rank in 2021-2022: Anastasia, Edward, Albert, Saint Alphonsus, John Brebeuf. 
2. but John Brebeuf grading rate 100%, Albert 84%, Anastasia 64%, and St. Edward not even half, only about 39%
3. and William don't have writing lesson, but they finish 5 reading practices per student, 
4. avg lessons: John Brebeuf 3.6, Edward 4.7, Albert 10.5, Anastasia 8, Saint Al 7 

### 4. School level word count and reading score

In [22]:
df[~df['lesson_type'].str.contains('Benchmark')].groupby(['school_year',
                                                          'school_name'])['word_count',
                                                                          'reading_score'].mean()#.to_excel(os.path.join(path, 'avg word count and reading.xlsx'))

word_count  \
school_year school_name                                               
2020-2021   Most Holy Redeemer School                           NaN   
            SS Faith, Hope, and Charity Catholic School  100.763158   
            Sacred Heart School                          144.333333   
            Saint Alphonsus Liguori School                42.407767   
            St Ann Grade School                           72.000000   
            St Gabriel School                            117.000000   
            St. Albert the Great Catholic School         173.280000   
            St. Ann School                               121.555556   
            St. Clement School                                  NaN   
            St. Edward School                            171.531008   
            St. John Brebeuf School                      450.962617   
            St. William School                                  NaN   
2021-2022   Sacred Heart School                          137.572581   
            Saint Alphonsus Liguori School                80.639073   
            St Anastasia School                          253.357983   
            St Thomas the Apostle School                  76.692308   
            St. Albert the Great Catholic School         188.771930   
            St. Ann School                                86.000000   
            St. Edward School                            172.601751   
            St. John Berchmans School                    275.679245   
            St. John Brebeuf School                      445.587121   
            St. William School                                  NaN   

                                                         reading_score  
school_year school_name                                                 
2020-2021   Most Holy Redeemer School                              NaN  
            SS Faith, Hope, and Charity Catholic School      76.888889  
            Sacred Heart School                              62.535211  
            Saint Alphonsus Liguori School                   65.348837  
            St Ann Grade School                              60.000000  
            St Gabriel School                                76.190476  
            St. Albert the Great Catholic School             69.508197  
            St. Ann School                                   57.435897  
            St. Clement School                                     NaN  
            St. Edward School                                76.356589  
            St. John Brebeuf School                          84.430769  
            St. William School                               61.806020  
2021-2022   Sacred Heart School                              68.717949  
            Saint Alphonsus Liguori School                   67.913935  
            St Anastasia School                              85.241581  
            St Thomas the Apostle School                     75.384615  
            St. Albert the Great Catholic School             85.092170  
            St. Ann School                                         NaN  
            St. Edward School                                77.469136  
            St. John Berchmans School                        72.170465  
            St. John Brebeuf School                          81.475000  
            St. William School                               66.538462

In [19]:
## check the class grade of the students in the schools for 2021-2022
## check if some schools have all lower grade students, that's why they have low word count
df[df['school_year'] != '2020-2021'].groupby(['school_name', 'grade'])['student_id'].nunique()

## grade level 5-8 for all students

school_name                           grade
Sacred Heart School                   3        17
                                      4        18
                                      5        17
                                      6        13
                                      7        13
                                      8        21
Saint Alphonsus Liguori School        4        11
                                      5        16
                                      6         9
                                      7        10
                                      8        16
St Anastasia School                   6        20
                                      7        31
                                      8        34
St Thomas the Apostle School          8        14
St. Albert the Great Catholic School  6        21
                                      7        16
                                      8        15
St. Ann School                        6        20
      

### Findings:

1. John Brebeuf highest word count, 445, next is Anastasia 253
2. Albert 188, they have the highest usage (in the regplot their data looks about right though), rs is high 85
3. Saint Al wc 80, rs 68
4. Edward 172 word count, 77 rs, but they only grade 40% of writing lesson so maybe that's why

## T-test

In [20]:
## Multiple Choice Score
# reshape the dataset so that 2020-2021 and 2021-2022 can be two columns side by side 

rd_pivot = df.pivot_table(index=['school_name', 'student_name','grade','lesson_type'], 
                          columns='school_year', values='reading_score')
rd_pivot = rd_pivot.reset_index()

rd_pivot = rd_pivot.dropna()

print(rd_pivot[['2020-2021','2021-2022']].describe())
print(ttest_rel(rd_pivot['2020-2021'], rd_pivot['2021-2022']))

## t-test results have shown that there is a difference between 2020-2021 and 2021-2022, and the writing scores improve

school_year   2020-2021   2021-2022
count        192.000000  192.000000
mean          72.905196   75.271267
std           21.023888   19.541065
min           10.000000   12.500000
25%           60.000000   62.500000
50%           80.000000   79.166667
75%           90.000000   90.000000
max          100.000000  100.000000
Ttest_relResult(statistic=-1.539547341523962, pvalue=0.12532577385835822)


In [21]:
## Writing Word Count
# reshape the dataset so that 2020-2021 and 2021-2022 can be two columns side by side 

wt_pivot = df.pivot_table(index=['school_name', 'student_name','grade','lesson_type'], 
                              columns='school_year', values='word_count')
wt_pivot = wt_pivot.reset_index()

wt_pivot = wt_pivot.dropna()

print(wt_pivot[['2020-2021','2021-2022']].describe())
print(ttest_rel(wt_pivot['2020-2021'], wt_pivot['2021-2022']))

## t-test results have shown that there is a difference between 2020-2021 and 2021-2022, and the writing scores improve

school_year   2020-2021   2021-2022
count        108.000000  108.000000
mean         276.257209  352.637963
std          152.272637  190.213485
min           18.000000   33.000000
25%          157.700000  194.333333
50%          296.375000  311.833333
75%          369.000000  513.250000
max          687.750000  994.000000
Ttest_relResult(statistic=-5.719075602329311, pvalue=9.81859467679596e-08)
